In [1]:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 10.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.1 MB/s eta 0:00:0000:01



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import os
import numpy as np
import tensorflow as tf

from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout
from keras import Model, callbacks

2023-04-20 08:58:00.558437: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
def init_vgg16(checkpoint_path:str=None) -> Model:
    """
    Creates a new instance of the VGG16 model with imageNet pretrained weights.
    
    args: 
        checkpoint_path: the path to the location of a tf checkpoint for resuming training
        
    returns:
        model: a tf.Model
    """

    # Download the model with weights pre-trained using ImageNet database
    vgg16 = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(224,224,3))
    
    
    # Freeze layers for training
    for layer in vgg16.layers:
        layer.trainable = False

    # Create a new 'top' of the model of fully-connected layers
    top_model = vgg16.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu', name="top_dense_1")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(512, activation='relu', name="top_dense_2")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(256, activation='relu', name="top_dense_3")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(128, activation='relu', name="top_dense_4")(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(1, activation='softmax', name="output" )(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=vgg16.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy'])
    
    if checkpoint_path is not None:
        model.load_weights(checkpoint_path)
    
    return model


In [4]:
def load_and_preprocess_img(path: str):

    img = load_img(path, target_size=(224, 224))
    ary = img_to_array(img)
    ary = np.expand_dims(ary, axis=0)
    ary = preprocess_input(ary)
    return ary[0]


In [5]:
from os import listdir

def load_data(path: str, label: int):

    X = []
    y = []

    for filename in listdir(path):
        x = load_and_preprocess_img(path + filename)
        X.append(x)
        y.append(label)

    return X, y


In [6]:
FILEPATH_POS = "./out_pos/"
FILEPATH_NEG = "./out_neg/"

X, y = load_data(FILEPATH_POS, 1)
X_neg, y_neg = load_data(FILEPATH_NEG, 0)

X.extend(X_neg)
y.extend(y_neg)

X = np.array(X)
y = np.array(y)

print(f"X: {X.shape}")
print(f"y: {y.shape}")


X: (6767, 224, 224, 3)
y: (6767,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [8]:
# Checkpoint during training
checkpoint_path = "classifier_chkpts/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model = init_vgg16(checkpoint_path)
model.summary()

2023-04-20 08:58:17.262946: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-20 08:58:17.265734: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-20 08:58:17.265856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

58889256/58889256 [==============================] - 5s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
batch_size = 32

# Create a callback that saves the model's weights
cp_callback = callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                        save_weights_only=True,
                                        save_freq=10*batch_size,
                                        verbose=1)

model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=200,
          validation_data=(X_test, y_test),
          callbacks=[cp_callback])

Epoch 1/200


2023-04-20 08:58:27.401110: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-04-20 08:58:28.841039: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-20 08:58:29.224788: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f8e7246bfb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-20 08:58:29.224809: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-04-20 08:58:29.227387: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-20 08:58:29.303012: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

142/142 [==============================] - 27s 163ms/step - loss: 0.2854 - binary_accuracy: 0.9173 - val_loss: 0.2996 - val_binary_accuracy: 0.9114
Epoch 2/200
142/142 [==============================] - 21s 145ms/step - loss: 0.2854 - binary_accuracy: 0.9173 - val_loss: 0.2996 - val_binary_accuracy: 0.9114
Epoch 3/200
 35/142 [======>.......................] - ETA: 10s - loss: 0.2797 - binary_accuracy: 0.9196
Epoch 3: saving model to classifier_chkpts/cp.ckpt
142/142 [==============================] - 22s 158ms/step - loss: 0.2854 - binary_accuracy: 0.9173 - val_loss: 0.2996 - val_binary_accuracy: 0.9114
Epoch 4/200
142/142 [==============================] - 21s 145ms/step - loss: 0.2854 - binary_accuracy: 0.9173 - val_loss: 0.2996 - val_binary_accuracy: 0.9114
Epoch 5/200
 71/142 [==============>...............] - ETA: 7s - loss: 0.2876 - binary_accuracy: 0.9164
Epoch 5: saving model to classifier_chkpts/cp.ckpt
142/142 [==============================] - 22s 156ms/step - loss: 0.2854 